In [ ]:
## 네이버 금융 개별종목 수집

In [ ]:
import pandas as pd

* 2020년 주요 상장주식
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

In [ ]:
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"


# 종목 URL 만들기
# url = f"https://finance.naver.com/item/sise.nhn?code={item_code}"
url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page=1"

In [ ]:
table = pd.read_html(url, encoding="cp949")
len(table)

In [ ]:
table[0]

In [ ]:
df = table[0].dropna()
df

## 페이지별 데이터 수집 함수 만들기

In [ ]:
def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    table = pd.read_html(url, encoding="cp949")
    df_day = table[0].dropna()
    return df_day

In [ ]:
# 함수가 잘 만들어졌는지 확인하기
page_no = 1
get_day_list(item_code, page_no)

## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 좀 2~3초 정도로 좀 더 길게 주세요. 

In [ ]:
import time
page_no = 1
item_list = []

while True:
    df_day = get_day_list(item_code, page_no)
    page_no += 1
    item_list.append(df_day)
    time.sleep(0.5)
    if len(df_day) != 10:
        break

## 수집한 데이터 하나의 데이터프레임으로 합치기

In [ ]:
df = pd.concat(item_list)
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

## 데이터프레임에 종목코드와 종목명을 추가하기

In [ ]:
df["종목코드"] = item_code
df["종목명"] = item_name

In [ ]:
df.head()

## 컬럼 순서 변경하기

In [ ]:
df = df[['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']]
df.head()

## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [ ]:
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

## 기술통계값 구하기

In [ ]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

## 최근 날짜 구해서 파일명 만들기

In [ ]:
date = df.iloc[0]["날짜"]
date

In [ ]:
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

## 파일로 저장하기

In [ ]:
# 파일로 저장하기 데이터프레임의 기본 index 는 저장하지 않도록 옵션 추가
df.to_csv(file_name, index=False)

In [ ]:
pd.read_csv(file_name)